In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier

# Import KMM method form adapt.instance_based module
from adapt.instance_based import KMM
import json


# with open('matbench_expt_is_metal.json', 'r') as json_file:
#     # Load the JSON data
#     data = json.load(json_file)

# df = pd.DataFrame(data['data'], columns = data['columns'])
# df = df.rename(columns={'composition': 'formula'})

In [3]:
# Define the file path
file_path = 'bandgap_target_clusters50.json'
ncluster=50
featurefile = 'bandgap_features.csv'
property_column='gap expt'

In [4]:
from matminer.featurizers.conversions import StrToComposition
import os

if not os.path.exists(featurefile):
    df1 = StrToComposition().featurize_dataframe(df, "formula")
    df1.head()

    from matminer.featurizers.composition import ElementProperty

    ep_feat = ElementProperty.from_preset(preset_name="magpie")
    df2 = ep_feat.featurize_dataframe(df1, col_id="composition")  # input the "composition" column to the featurizer
    df2.head()

    #y = df2['gfa'].values
    excluded = ["formula", "composition", "formula", property_column]

    #X = df2.drop(excluded, axis=1)
    df2.to_csv(featurefile)
else:  #load the features
    df2 = pd.read_csv(featurefile)


In [5]:
y = df2[property_column].values
excluded = ["formula", "composition", "formula", property_column]
X = df2.drop(excluded, axis=1)

In [6]:
# print(X.head)
# print(X.shape)
# print(X.iloc[0])
# print(y)

In [7]:
# from pymatgen.core import Composition

# target_id_list = []
# for i, s in enumerate(df2["formula"]):
#     if "Cu" in Composition(s).as_dict().keys() and "Li" in Composition(s).as_dict().keys():
#         target_id_list.append(i)
# print(target_id_list)

In [8]:


# Read the JSON file
with open(file_path, 'r') as file:
    cluster_ids = json.load(file)

# Access the data from the JSON file
target_id_list = cluster_ids["2"]
print(target_id_list)

[1841, 1980, 2007, 2099, 2123, 2127, 2135, 2152, 2153, 2157, 2159, 2160, 2180, 2188, 2198, 2200, 2405, 2416, 2417, 2418, 2419, 2420, 2429, 2441, 2442, 2457, 2468, 2487, 2488, 2495, 2585, 2586, 2588, 2590, 2592, 2593, 2597, 2599, 2606, 2609, 2610, 2611, 2612, 2614, 2615, 2616, 2619, 2620, 2622, 2625, 2627, 2632, 2633, 2634, 2637, 2641, 2642, 2648, 2649, 2653, 2656, 2658, 2664, 2665, 2666, 2682, 2696, 2702, 2712, 2723, 2725, 2730, 2731, 2765, 2854, 2909, 2924, 2934, 2936, 2941, 2943, 2954, 2955, 2956, 2977, 2986, 2992, 2993, 2995, 2997, 3070, 3074, 3075, 3077, 3090, 3093, 3097, 3098, 3105, 3106, 3109, 3110, 3115, 3124, 3130, 3135, 3146, 3154, 3201, 3262]


In [ ]:
from adapt.instance_based import LDM

num_improvement = 0
num_worse = 0
average_improvement = 0
average_worse = 0
baseline_scores=[]
adapt_scores=[]

for i, cluster_id in enumerate(range(0,ncluster)):
    target_id_list = cluster_ids[str(cluster_id)]

    Xt = X.iloc[target_id_list]
    yt = df2["gap expt"].iloc[target_id_list].values

    Xs = X.drop(target_id_list)
    ys = df2["gap expt"].drop(target_id_list).values
    
    src_only = RandomForestRegressor(n_estimators=100, max_depth=10)
    print(f"{i}\t",end="")
    src_only.fit(Xs, ys)

    score = src_only.score(Xt, yt)
    print("src_only score: ", score, end="")
    baseline_scores.append(score)
    
    # Instantiate a KMM model : estimator and target input
    # data Xt are given as parameters with the kernel parameters
#     adapt_model = KMM(
#         estimator=RandomForestRegressor(n_estimators=100, max_depth=10),
#         Xt=Xt,
#         kernel="rbf",  # Gaussian kernel
#         gamma=1.,     # Bandwidth of the kernel
#         verbose=0,
#         random_state=0
#     )
    adapt_model = LDM(
        estimator=RandomForestRegressor(n_estimators=100, max_depth=10),
        Xt=Xt,
        verbose=0,
        random_state=0
    )    

    # Fit the model.
    adapt_model.fit(Xs, ys)

    # Get the score on target data
    adapt_score = adapt_model.score(Xt, yt)
    print("\tadapt_model score: ", adapt_score)
    adapt_scores.append(adapt_score)
    
    if adapt_score < score:
        num_improvement += 1
        a = -(score-adapt_score)/score
        average_improvement += a
    else: 
        num_worse += 1
        w = (adapt_score-score)/score
        average_worse += w


print(f"average improvement: {(average_improvement/ncluster)*100} %")
print(f"average worse: {(average_worse/ncluster)*100} %")
print(f"baseline avg. accuracy:{sum(baseline_scores)/len(baseline_scores)}")
print(f"adapt avg. accuracy:{sum(adapt_scores)/len(adapt_scores)}")
print(f"num_improvement:{num_improvement}")
print(f"num_worse:{num_worse}")


0	src_only score:  0.3962544935025284     pcost       dcost       gap    pres   dres   k/t
 0:  6.5320e-12  2.2631e-09  6e+03  2e-05  3e+02  1e+00
 1:  8.8251e-01  9.4360e-01  3e+02  1e-06  2e+01  1e-01
 2:  1.6253e+00  1.6987e+00  2e+02  4e-07  6e+00  9e-02
 3:  3.4270e+00  3.5397e+00  2e+02  2e-07  3e+00  1e-01
 4:  7.4894e+00  7.6851e+00  4e+02  2e-07  2e+00  2e-01
 5:  1.5758e+01  1.6083e+01  6e+02  1e-07  2e+00  3e-01
 6:  3.3168e+01  3.3688e+01  1e+03  9e-08  1e+00  5e-01
 7:  6.6426e+01  6.7220e+01  2e+03  7e-08  1e+00  8e-01
 8:  1.1717e+02  1.1828e+02  3e+03  6e-08  9e-01  1e+00
 9:  2.0881e+02  2.1037e+02  6e+03  5e-08  7e-01  2e+00
10:  3.6316e+02  3.6530e+02  9e+03  4e-08  6e-01  2e+00
11:  7.0675e+02  7.1012e+02  1e+04  3e-08  5e-01  3e+00
12:  1.0303e+03  1.0344e+03  2e+04  3e-08  4e-01  4e+00
13:  1.6686e+03  1.6741e+03  3e+04  2e-08  3e-01  5e+00
14:  2.8001e+03  2.8074e+03  4e+04  2e-08  2e-01  7e+00
15:  3.9965e+03  4.0052e+03  5e+04  1e-08  2e-01  9e+00
16:  5.1711e+

In [ ]:
#!pip3 install adapt


In [ ]:
# from sklearn.model_selection import train_test_split


# Xs, Xt, ys, yt = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
'''



# np.random.seed(0)

# # Create source dataset (Xs ~ N(-1, 2))
# # ys = 1 for ys in [-1, 1] else, ys = 0
# Xs = np.random.randn(1000, 1)*2-1
# ys = (Xs[:, 0] > -1.) & (Xs[:, 0] < 1.)

# # Create target dataset (Xt ~ N(1, 2)), yt ~ ys
# Xt = np.random.randn(1000, 1)*2+1
# yt = (Xt[:, 0] > -1.) & (Xt[:, 0] < 1.)

# Instantiate and fit a source only model for comparison
src_only = MLPClassifier(alpha=1, max_iter=1000)
print("TRAIN")
src_only.fit(Xs, ys)

print("src_only score: ", src_only.score(Xt, yt))

# Instantiate a KMM model : estimator and target input
# data Xt are given as parameters with the kernel parameters
adapt_model = KMM(
    estimator=MLPClassifier(alpha=1, max_iter=1000),
    Xt=Xt,
    kernel="rbf",  # Gaussian kernel
    gamma=1.,     # Bandwidth of the kernel
    verbose=0,
    random_state=0
)

# Fit the model.
adapt_model.fit(Xs, ys)

# Get the score on target data
print("adapt_model score: ", adapt_model.score(Xt, yt))

'''

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# weights = adapt_model.predict_weights()
# Xs_weighted = np.random.choice(Xs.ravel(), 1000, p=weights/weights.sum())
# limit_noadapt = src_only.intercept_ / src_only.coef_
# limit_adapt = adapt_model.estimator_.intercept_ / adapt_model.estimator_.coef_
# acc_noadapt = src_only.score(Xt, yt)
# acc_adapt = adapt_model.estimator_.score(Xt, yt)

# k1 = sns.kdeplot(Xs.ravel(), color="blue", bw_method=0.5, label="Source")
# k2 = sns.kdeplot(Xt.ravel(), color="red", bw_method=0.5, label="Target")
# k3 = sns.kdeplot(Xs_weighted, color="blue", ls="--",
#                  bw_method=0.5, label="Reweighted source")

# plt.plot([limit_noadapt[0, 0]]*2, [0, 1], ls=":", c="k")
# plt.plot([limit_adapt[0, 0]]*2, [0, 1], ls="--", c="k")
# plt.fill_between([-1, 1], [0, 0], [1, 1], alpha=0.2, color="k")
# plt.fill_between([-9, 9], [0, 0], [1, 1], alpha=0.05, color="k")
# plt.text(-6, 0.2, "_", fontsize=20)
# plt.text(-0.45, 0.24, "+", fontsize=20)
# plt.text(5.5, 0.2, "_", fontsize=20)
# plt.ylim(0, 0.3); plt.xlim(-9, 9)

# plt.text(-8.5, 0.25,
#          (r"$\cdots$ No adapt : Acc = %.2f"%acc_noadapt + "\n" +
#           r"--- Adapt : Acc = %.2f"%acc_adapt),
#          bbox=dict(boxstyle='round', fc='w', ec="gray"))
# plt.legend(); plt.show();

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import make_pipeline
# from sklearn.datasets import make_moons, make_circles, make_classification
# from sklearn.neural_network import MLPClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.inspection import DecisionBoundaryDisplay

# classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
#     SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1, max_iter=1000),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis(),
# ]